# Fully-saturated TWFE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
import gpboost as gpb
from patsy import dmatrix

In [2]:
path = '../../../../data/processed/df_filtered.xlsx'
df = pd.read_excel(path)
df['log_pop'] = np.log(df['populations'])
df.dropna(subset=['log_pop'], inplace=True)
df

,island_ids,years,region_codes,prefecture_codes,populations,treatment_group,treatment_group_mainland,year_bridges_opened,after_treated,year_connect_mainland,...,area_km2,distance_m,elementary_school,junior_high_school,high_school,hospital,population_change_rate,control_group,time_since_intervention,log_pop
5,67,1995,32202,32,426.0,1,1,1999,0,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,NaN,0,-4,6.054439
6,67,2000,32202,32,324.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-23.943662,0,1,5.780744
7,67,2005,32202,32,302.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-6.790123,0,6,5.710427
8,67,2010,32202,32,265.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-12.251656,0,11,5.579730
9,67,2015,32202,32,197.0,1,1,1999,1,1999,...,0.28,615.0,NaN,NaN,NaN,NaN,-25.660377,0,16,5.283204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,226,2000,39424,39,398.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-4.784689,0,43,5.986452
1896,226,2005,39424,39,365.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-8.291457,0,48,5.899897
1897,226,2010,39424,39,343.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-6.027397,0,53,5.837730
1898,226,2015,39424,39,284.0,1,1,1957,1,1957,...,0.56,110.0,NaN,NaN,NaN,NaN,-17.201166,0,58,5.648974


In [3]:
y = df['log_pop']
group = df[['island_ids', 'years']]
X = dmatrix("C(year_bridges_opened) : C(time_since_intervention) : treatment_group - 1", df)

gp_model = gpb.GPModel(group_data=group, likelihood="gaussian")
gp_model.fit(y, X, params={'std_dev': True})
gp_model.summary()

[GPBoost] [Warning] The linear regression covariate data matrix (fixed effect) is rank deficient. This is not necessarily a problem when using gradient descent. If this is not desired, consider dropping some columns / covariates 


[GPBoost] [Warning] Sample size too small to calculate standard deviations for coefficients
Model summary:
 Log-lik      AIC      BIC
-1931.82 11933.63 33870.38
Nb. observations: 1697
Nb. groups: 94 (island_ids), 30 (years)
-----------------------------------------------------
Covariance parameters (random effects):
            Param.  Std. dev.
Error_term  0.4336     0.0155
island_ids  0.8208     0.1255
years       1.6159     0.4295
-----------------------------------------------------
Linear regression coefficients (fixed effects):
                                                    Param.  Std. dev.  \
C(year_bridges_opened)[0]:C(time_since_interven...     0.0        NaN   
C(year_bridges_opened)[1889]:C(time_since_inter...     0.0        NaN   
C(year_bridges_opened)[1935]:C(time_since_inter...     0.0        NaN   
C(year_bridges_opened)[1950]:C(time_since_inter...     0.0        NaN   
C(year_bridges_opened)[1951]:C(time_since_inter...     0.0        NaN   
...                   

In [6]:
df['years'].unique()

array([1995, 2000, 2005, 2010, 2015, 2020, 1970, 1975, 1980, 1985, 1990,
       2001, 2002, 2003, 2004, 2006, 2007, 2008, 2009, 2011, 2012, 2013,
       2014, 2016, 2017, 2018, 2019, 1955, 1960, 1965])